In [1]:
!pip install transformers -q
!pip install streamlit -q
!pip install anthropic -q
!pip install pypdf -q
!pip install pyngrok -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 891.5/891.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 2.1 MB/s eta 0:00:00


In [21]:
%%writefile app.py
import streamlit as st
import requests, os
from anthropic import Anthropic
from pypdf import PdfReader

client = Anthropic(
    api_key=''
)
MODEL_NAME = 'claude-3-5-sonnet-20240620'

# Intro Text
st.markdown('# PDF summarization using Generative AI')
st.header('''PDF size must be below 200MB. Only 1 PDF per time, time will vary depending on size''')

input_file = st.file_uploader('Upload a PDF file')

options = st.radio(
    'What type of summary would you like?',
    ['High-Level','Technical'],
    captions=[
       'A summary that leaves out any technical terms for a simpler summary',
       'A detailed summary that includes the technical aspects of the paper',
    ],
    index=None
)

# Save uploaded file
if input_file:
  with open(os.path.join(input_file.name),'wb') as f:
        f.write(input_file.getbuffer())
  st.success('Saved File')

st.markdown('# Summary')

def read_pdf(pdf_file):
  reader = PdfReader(pdf_file)
  content = ''.join(page.extract_text() for page in reader.pages)
  return content

def get_completion(client, prompt):
  return client.messages.create(
      model=MODEL_NAME,
      max_tokens=2048,
      messages=[{
          "role": 'user', "content":  prompt
      }]
  ).content[0].text

if input_file and options == 'High-Level':

  text = read_pdf(input_file)
  completion = get_completion(client,
    f"""Here is an academic paper: <paper>{text}</paper>

  Please do the following:
  1.Provide a high-level overview of the research paper in such a way that the reader won't need to read the whole paper.
  2.Assume you are an academic who wants to find more research papers like this one. Return a list of 5 research papers that are similar to
  this paper. Sort the list in descending order from most relevant to least relevant.
  """
  )
  st.success('Summary ready!')
  st.text_area(label = '', value = completion, placeholder = 'Please upload a PDF to summarise', height = 400)
  st.markdown('')

elif input_file and options == 'Technical':

  text = read_pdf(input_file)
  completion = get_completion(client,
    f"""Here is an academic paper: <paper>{text}</paper>

  Please do the following:
  1.Provide a detailed technical summary of the research paper in such a way that the reader won't need to read the whole paper.
  2.Assume you are an academic who wants to find more research papers like this one. Return a list of 5 research papers that are similar to
  this paper. Sort the list in descending order from most relevant to least relevant.
  """
  )
  st.success('Summary ready!')
  st.text_area(label = '', value = completion, placeholder = 'Please upload a PDF to summarise', height = 400)
  st.markdown('')

else:
  st.text_area(label = '',value = 'Please upload a PDF to summarise', placeholder = 'Please upload a PDF to summarise', height = 400)

Overwriting app.py


Use the below to pass "number" to prompt. Assume you are an academic who wants to find more research papers like this one. Return a list of <number>{number}</number> research papers that are similar to this paper. Sort the list in descending order from most relevant to least relevant

In [ ]:
import streamlit as st

number = st.number_input(
    "Insert a number", value=None, placeholder="Type a number..."
)
st.write("The current number is ", number)

In [22]:
from pyngrok import ngrok

# Auth Token
ngrok.set_auth_token('')

# Start Streamlit server and set port number
!nohup streamlit run app.py --server.port 5011 &

# Start ngrok tunnel
ngrok_tunnel = ngrok.connect(addr='5011', proto='http', bind_tls=True)
print('Tunnel URL:', ngrok_tunnel.public_url)

nohup: appending output to 'nohup.out'
Tunnel URL: https://b0a6-34-70-3-42.ngrok-free.app


In [23]:
# Shut down tunnel
print(ngrok.get_tunnels())
ngrok.disconnect(ngrok_tunnel.public_url)
print(ngrok.get_tunnels())

[<NgrokTunnel: "https://b0a6-34-70-3-42.ngrok-free.app" -> "http://localhost:5011">]
[]


### TASK
  You are an expert in text analysis and summarisation with research papers. Your task is to analyse a research paper I'm going to give you
  and provide a detailed summary that is most representative of its content. You must use the Key Concept Clarity method to read the paper and
  highlight all of the key talking points of the paper.
  ### INPUTS
  Research Paper: {input_file}
  ### OUTPUT
  The output should be structured so as to identify and concisely explain the main ideas in the document. Each section should consist of the
  name of the talking point, a detailed definition of the talking point and a short summary of the talking point.
  Aim to provide at least four talking points. Here is how each talking point should be structured:
  Name of the talking point
  Explanation: Define in detail what is the talking point
  Overview: Provide a short summary of the talking point including its challenges ,if any, and findings.
  '''